# Limits with toy experiments

_Valerio Ippolito - INFN Sezione di Roma_

ROOT provides a nice example of how to run limits with multiple possible test statistics, possibly using toy experiments instead of asymptotic formulae.

In [ ]:
!ls $ROOTSYS/tutorials/roostats/StandardHypoTestInvDemo.C

In [ ]:
!cp $ROOTSYS/tutorials/roostats/StandardHypoTestInvDemo.C .

In [ ]:
gROOT->LoadMacro("StandardHypoTestInvDemo.C+")

In [ ]:
inputFile = TString("../ws/ATLASIT_prova_combined_ATLASIT_prova_model.root");
workspaceName = TString("combined");
modelConfigName = TString("ModelConfig");
modelBName = TString(""); // bkg-only model, will be created by setting POI to zero if doesn't exist
dataName = TString("obsData"); 

Which test statistics do we want?
- `0`: LEP
- `1`: Tevatron
- `2`: Profile Likelihood two sided
- `3`: Profile Likelihood one sided (i.e. = 0 if mu < mu_hat)
- `4`: Profile Likelihood signed ( pll = -pll if mu < mu_hat)
- `5`: Max Likelihood Estimate as test statistic
- `6`: Number of observed event as test statistic

In which way do we want to calculate the test statistics distribution?

- `0`: Freq calculator
- `1`: Hybrid calculator
- `2`: Asymptotic calculator
- `3`: Asymptotic calculator using nominal Asimov data sets (not using fitted parameter values but nominal ones)

In [ ]:
testStatType = 3;
calculatorType = 2;

Do we want to use CLs to evaluate the limit?

In [ ]:
useCLs = kTRUE;

This is usually the delicate part: the POI scan which is used to determine the POI interval from the crossing of the test statistics value with the desired horizontal line (0.05 for 95% CL). If the scanned range doesn't contain the intersection, everything will result in a poor extrapolation, and limits will be meaningless...

In [ ]:
npoints = Int_t(6);
poimin = Double_t(0);
poimax = Double_t(20);

If you pick the frequentist calculator, you need to specify how many toys to use:

In [ ]:
ntoys = 1000;

Let's run:

In [ ]:
StandardHypoTestInvDemo(inputFile, workspaceName, modelConfigName, modelBName, dataName,
                        calculatorType, testStatType,
                        useCLs,
                        npoints, poimin, poimax,
                        ntoys,
                        false, 0) // leave these two options like this

Let's open the output

In [ ]:
!ls -lhtr

Make sure you select the correct file - it's named based on the features of how you ran the limit, and a typical mistake is that of keeping opening the same old file.

In [ ]:
f = new TFile("Asym_CLs_grid_ts3_ATLASIT_prova_combined_ATLASIT_prova_model.root");
f->ls();

You may re-retrieve the results with

In [ ]:
result = dynamic_cast<RooStats::HypoTestInverterResult*>(f->Get("result_mu_ttH"));
cout << "Lower limit: " <<  result->LowerLimit() << " +/-" << result->LowerLimitEstimatedError() << endl;
cout << "Upper limit: " <<  result->UpperLimit() << " +/-" << result->UpperLimitEstimatedError() << endl;
cout << "Median expected upper limit: " << result->GetExpectedUpperLimit(0) << endl;
cout << "Median +1sigma expected upper limit: " << result->GetExpectedUpperLimit(+1) << endl;
cout << "Median -1sigma expected upper limit: " << result->GetExpectedUpperLimit(-1) << endl;

As you see, the lower limit is invalid (we ran an upper limit evaluation!).

Let's plot the POI scan from which the limit is evaluated:

In [ ]:
plot = new HypoTestInverterPlot("plot", "plot", result);

In [ ]:
c = new TCanvas();
plot->Draw("CLb 2CL"); // plot all and Clb
c->Draw();

As you can see, we picked the wrong POI scan (compare with the outputs of `asymptotics.ipynb`): it's very hard to see where the graph of the observed CLs intersects with the 0.05 horizontal line...

## Exercises

- How meaningless! Let's try again with a better setting of the POI scan...
- Once you got the correct scan, try moving to the frequentist calculator (i.e. toys, i.e. `calculatorType=0`)